In [ ]:
!pip install pandas numpy tqdm transformers datasets sklearn

In [2]:
#https://huggingface.co/transformers/master/custom_datasets.html

In [1]:
import pandas as pd
import numpy as np
import pathlib
from random import randint
import pickle
import time

import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification
from transformers import TFTrainingArguments, TrainingArguments
from transformers import Trainer, TFTrainer

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


from tqdm import tqdm
tqdm.pandas()

import transformers
transformers.logging.set_verbosity_info()


In [2]:
#finetuning with custom datasets:
#https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python
#https://huggingface.co/transformers/custom_datasets.html

# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "distilbert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.5.1",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe

In [4]:
df = pd.read_csv("output.csv")

In [5]:
df = df.drop(["creator_id", "creator_real_name", "creator_name", "creator_email", "creator_nick"], axis=1)

In [6]:
df.head()

,id,summary,status,resolution,description,dupe_of,op_sys,platform,priority,component,severity,creation_time,last_change_time,keywords,resolution.1,is_open
0,10954,Dialup properties needs to be exposed in prefs,RESOLVED,WONTFIX,The dialup properties of the profile should be...,NaN,All,All,P3,Preferences,normal,1999-07-30T22:55:51Z,2008-05-14T18:44:15Z,[],WONTFIX,False
1,14871,[Find] Find whole word only,RESOLVED,DUPLICATE,"Please add ""Match Whole Word Only"" option to b...",269442.0,All,All,--,General,minor,1999-09-24T21:49:34Z,2017-01-28T19:55:08Z,['helpwanted'],DUPLICATE,False
2,19118,Plug-In Manager (ui for choosing mimetype-plug...,RESOLVED,WONTFIX,I would really like a plug-in manager for my b...,NaN,All,All,--,Preferences,normal,1999-11-17T22:58:26Z,2018-03-05T20:10:55Z,[],WONTFIX,False
3,21482,Improvement to Save File dialog: folder based ...,NEW,NaN,Perhaps this really should be in the standardi...,NaN,All,All,P3,File Handling,normal,1999-12-11T04:13:35Z,2019-01-03T17:33:57Z,[],NaN,True
4,23207,"Options in Save As (location of saved images, ...",NEW,NaN,Save as... feature should be extended in follo...,NaN,Unspecified,Unspecified,P5,File Handling,normal,2000-01-06T03:19:51Z,2020-12-11T16:34:00Z,[],NaN,True


In [7]:
tokenizer(df["description"].iloc[0], truncation=True, padding=True)

{'input_ids': [101, 1996, 13764, 6279, 5144, 1997, 1996, 6337, 2323, 2022, 6086, 1999, 1996, 3653, 10343, 9320, 2061, 1996, 5310, 2038, 2019, 3733, 2126, 2000, 19933, 2068, 1012, 1996, 2069, 2060, 4522, 2052, 2022, 2000, 2191, 2111, 2175, 2000, 1996, 6337, 3208, 2000, 10086, 2068, 1010, 2021, 2057, 2123, 1005, 1056, 2031, 2008, 1999, 2173, 2593, 1012, 2292, 1005, 1055, 3046, 1996, 3653, 10343, 5997, 3921, 2034, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
df["description"].isnull().sum()

2973

In [9]:
df.dropna(subset=['description'], inplace=True)

In [10]:
df["component"].isnull().sum()

0

In [11]:
import re

counter = 0

def slice_row(row):
    global counter
    if pd.isnull(row):
        return row
#    if counter > 10:
#        return row
    counter += 1
    
    #start = row.find("User-Agent:")
    start = re.search(r'User.Agent:', row)
    if start is None:
        return row
    start = start.start()
    while start >= 0:
        end = row.find("\n", start)
#        print(f"----------- {counter} -------------")
#        print(f"removing ua: {start} - {end}")
#        print(row)
        if end < 0:
            end = len(row)
        if end == 0:
            break
        row_start = row[:start]
        row_end = row[end:]
        row = row_start+" "+row_end
        #print(row)
        #start = row.find("User-Agent:", start)
        start = re.search(r'User.Agent:', row)
        if start is None:
            break
        start = start.start()
        
    start = row.find("Build Identifier:")
#    start = re.search(r'User.Agent:', row).start()
    #print("b")
    while start >= 0:
        end = row.find("\n", start)
        if end < 0:
            end = len(row)
        if end == 0:
            break
        row_start = row[:start]
        row_end = row[end:]
        row = row_start+" "+row_end
        start = row.find("Build Identifier:")
    #print("c")
    return row
        
df["description"] = df["description"].progress_apply(slice_row)#.str.startswith("User-Agent:").sum()
df["summary"] = df["summary"].progress_apply(slice_row)#.str.startswith("User-Agent:").sum()


100%|██████████| 187790/187790 [00:00<00:00, 581487.27it/s]


In [12]:
#df["text_feature"] = df["summary"]+"\n"+df["description"]
df['text_feature'] = df[["summary", "description"]].apply(lambda row: '\n'.join(row.values.astype(str)), axis=1)

In [13]:
components_to_remove = ['System Add-ons: Off-train Deployment',
                         'Activity Streams: General',
                         'Activity Streams: Server Operations',
                         'Activity Streams: Timeline',
                         'Untriaged']

for comp in components_to_remove:
    print(f"dropping rows for component {comp}")
    df = df[df["component"] != comp]
    

dropping rows for component System Add-ons: Off-train Deployment
dropping rows for component Activity Streams: General
dropping rows for component Activity Streams: Server Operations
dropping rows for component Activity Streams: Timeline
dropping rows for component Untriaged


In [14]:
num_labels = df["component"].nunique()
num_labels

47

In [15]:
#texts = list(df["description"])
texts = list(df["text_feature"])
labels_text = df["component"]

In [16]:

le = LabelEncoder()
le.fit(labels_text)
le.classes_

array(['Address Bar', 'Bookmarks & History', 'Disability Access',
       'Distributions', 'Downloads Panel', 'Enterprise Policies',
       'Extension Compatibility', 'File Handling', 'Firefox Accounts',
       'Firefox Monitor', 'Foxfooding', 'General', 'Headless',
       'Installer', 'Keyboard Navigation', 'Launcher Process', 'Menus',
       'Messaging System', 'Migration', 'New Tab Page',
       'Nimbus Desktop Client', 'Normandy Client', 'Normandy Server',
       'PDF Viewer', 'Page Info Window', 'Pioneer', 'Pocket',
       'Preferences', 'Private Browsing', 'Protections UI',
       'Remote Settings Client', 'Screenshots', 'Search', 'Security',
       'Session Restore', 'Shell Integration', 'Site Identity',
       'Site Permissions', 'Sync', 'Tabbed Browser', 'Theme',
       'Toolbars and Customization', 'Top Sites', 'Tours', 'Translation',
       'WebPayments UI', 'about:logins'], dtype=object)

In [17]:
le.classes_.shape

(47,)

In [18]:
labels = le.transform(labels_text)

In [19]:
np.unique(labels).shape

(47,)

In [20]:
le.transform(["Pocket"])

array([26])

In [21]:
le.inverse_transform([26])

array(['Pocket'], dtype=object)

In [22]:
print("seeds = [", end='')
max = 100000000
for x in range(10):
    if x > 0:
        print(", ", end='')
    seed = randint(0, max)
    print(f"{seed}", end='')
print("]")

seeds = [30770949, 54566994, 87210090, 81844137, 26293858, 59823568, 28014493, 91140897, 75347805, 14034667]


In [23]:
#seeds = [10565036, 95525152, 81064673, 8702320, 7213066, 17350873, 75652258, 30526287, 18683835, 11597371]
#only use one for github example
seeds = [10565036]

In [24]:

pathlib.Path('datasets').mkdir(parents=True, exist_ok=True)


In [25]:
pathlib.Path(f"datasets-notriage/{seeds[0]}.pickle").exists()

True

In [26]:

for seed in tqdm(seeds):
    if pathlib.Path(f"datasets-notriage/{seed}.pickle").exists():
        print(f"Dataset for seed {seed} exists. Skipping creation.")
        continue
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1, random_state=seed, stratify=labels)
    train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=seed, stratify=train_labels)
    dataset = {"train_texts": train_texts, "test_texts": test_texts, "val_texts": val_texts,
              "train_labels": train_labels, "test_labels": test_labels, "val_labels": val_labels}
    with open(f'datasets-notriage/{seed}.pickle', 'wb') as handle:
        pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


100%|██████████| 1/1 [00:00<00:00, 6636.56it/s]

Dataset for seed 10565036 exists. Skipping creation.


In [27]:
!ls -l datasets

total 1581012
-rw-r--r-- 1 root root 161885167 May 11 20:18 10565036.pickle
-rw-r--r-- 1 root root 161885131 May 12 05:01 11597371.pickle
-rw-r--r-- 1 root root 161885158 May 12 05:01 17350873.pickle
-rw-r--r-- 1 root root 161885158 May 12 05:01 18683835.pickle
-rw-r--r-- 1 root root 161885122 May 12 05:01 30526287.pickle
-rw-r--r-- 1 root root 161885095 May 12 05:01 7213066.pickle
-rw-r--r-- 1 root root 161885149 May 12 05:01 75652258.pickle
-rw-r--r-- 1 root root 161885059 May 12 05:01 81064673.pickle
-rw-r--r-- 1 root root 161885131 May 12 05:01 8702320.pickle
-rw-r--r-- 1 root root 161885086 May 12 05:01 95525152.pickle


In [28]:
tokenizer.model_max_length

512

In [29]:
pathlib.Path('encoded-datasets-notriage').mkdir(parents=True, exist_ok=True)


In [30]:
%%time
#encoded_datasets = []
for seed in tqdm(seeds):
    if pathlib.Path(f"encoded-datasets-notriage/{seed}.pickle").exists():
        print(f"Encoded dataset for seed {seed} exists. Skipping creation.")
        continue

    with open(f'datasets-notriage/{seed}.pickle', 'rb') as handle:
        dataset = pickle.load(handle)
        print(f"labels: {np.unique(dataset['test_labels'])}")
        
    train_encodings = tokenizer(dataset["train_texts"], truncation=True, padding=True)
    val_encodings = tokenizer(dataset["val_texts"], truncation=True, padding=True)
    test_encodings = tokenizer(dataset["test_texts"], truncation=True, padding=True)
    encodings = {"train_encodings": train_encodings, "val_encodings": val_encodings, "test_encodings": test_encodings,
                "train_labels": dataset["train_labels"], "val_labels": dataset["val_labels"], "test_labels": dataset["test_labels"]}

    with open(f'encoded-datasets-notriage/{seed}.pickle', 'wb') as handle:
        pickle.dump(encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

100%|██████████| 1/1 [00:00<00:00, 7476.48it/s]

Encoded dataset for seed 10565036 exists. Skipping creation.
CPU times: user 1.9 ms, sys: 0 ns, total: 1.9 ms
Wall time: 1.46 ms


In [31]:
pathlib.Path('tf-datasets').mkdir(parents=True, exist_ok=True)

In [32]:
#https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

In [33]:
pathlib.Path('trained-models-notriage').mkdir(parents=True, exist_ok=True)

In [34]:
def create_tf_dataset(seed, postfix=""):
    with open(f'encoded-datasets{postfix}/{seed}.pickle', 'rb') as handle:
        encoded_dataset = pickle.load(handle)
    
    print("creating tf train dataset")
    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(encoded_dataset["train_encodings"]),
        encoded_dataset["train_labels"]
    ))

    print("creating tf val dataset")
    val_dataset = tf.data.Dataset.from_tensor_slices((
        dict(encoded_dataset["val_encodings"]),
        encoded_dataset["val_labels"]
    ))

    print("creating tf test dataset")
    test_dataset = tf.data.Dataset.from_tensor_slices((
        dict(encoded_dataset["test_encodings"]),
        encoded_dataset["test_labels"]
    ))
    
    tf_dataset = {"train_dataset": train_dataset, "val_dataset": val_dataset, "test_dataset": test_dataset}
    return tf_dataset

In [35]:
!rm -rf results-notriage

In [36]:
#https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python

def train_from_dataset(seed, tf_dataset):
    training_args = TFTrainingArguments(
        output_dir=f'./results-notriage/{seed}',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir=f'./logs/{seed}',            # directory for storing logs
        logging_steps=500,
        load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
        #https://huggingface.co/transformers/main_classes/trainer.html#transformers.TFTrainingArguments
        #setting evaluation_strategy=steps with small number of steps -> extremely slow training
        evaluation_strategy="steps",     # steps = evaluate each `logging_steps`, epoch=end of epoch
        disable_tqdm=False, #should enable some kind of progress bar..
    )

    with training_args.strategy.scope():
        model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

    #https://huggingface.co/transformers/main_classes/trainer.html#transformers.TFTrainer
    trainer = TFTrainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=tf_dataset["train_dataset"],         # training dataset
        eval_dataset=tf_dataset["val_dataset"],            # evaluation dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    )

    train_output = trainer.train()
    eval_output = trainer.evaluate()
    
    #https://github.com/huggingface/transformers/issues/7849
    model.save_pretrained(f"trained-models-notriage/distilbert-base-uncased-{seed}")

In [37]:
%%time

for seed in tqdm(seeds):
    print(f"creating datasets for {seed}")
    tf_dataset = create_tf_dataset(seed, "-notriage")

    secs = 30
    print(f"sleeping for {secs}s.")
    time.sleep(secs)

    print(f"training for {seed}")
    train_from_dataset(seed, tf_dataset)

    print(f"sleeping for {secs}s.")
    time.sleep(secs)

    #this did not work.. internal error saving dataset https://stackoverflow.com/questions/56862492/how-to-save-tf-data-dataset-object
    #with open(f'tf-datasets/{seed}.pickle', 'wb') as handle:
    #    pickle.dump(tf_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    #tf_datasets.append(tf_dataset)
    #print(f"{top5_correct/len(test_labels}")


  0%|          | 0/1 [00:00<?, ?it/s]

creating datasets for 10565036
creating tf train dataset
creating tf val dataset
creating tf test dataset
sleeping for 30s.


The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy


training for 10565036


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
   

...

{'eval_loss': 1.654374416519261, 'eval_accuracy': 0.5937172431865828, 'epoch': 0.06555657532450505, 'step': 500}
{'loss': 2.5419693, 'learning_rate': 5e-05, 'epoch': 0.06555657532450505, 'step': 500}
Saving checkpoint for step 500 at ./results-notriage/10565036/checkpoint/ckpt-1
***** Running Evaluation *****
  Num examples in dataset = 30507
  Num examples in used in evaluation = 30528
  Batch size = 64
{'eval_loss': 1.3821111715064858, 'eval_accuracy': 0.6367596960167715, 'epoch': 0.1311131506490101, 'step': 1000}
{'loss': 2.0303624, 'learning_rate': 4.888298e-05, 'epoch': 0.1311131506490101, 'step': 1000}
Saving checkpoint for step 1000 at ./results-notriage/10565036/checkpoint/ckpt-2
***** Running Evaluation *****
  Num examples in dataset = 30507
  Num examples in used in evaluation = 30528
  Batch size = 64
{'eval_loss': 1.2780508365271226, 'eval_accuracy': 0.6480935534591195, 'epoch': 0.19666972597351515, 'step': 1500}
{'loss': 1.7955428, 'learning_rate': 4.776596e-05, 'epoch': 

{'eval_loss': 0.8574571909394654, 'eval_accuracy': 0.7515723270440252, 'epoch': 3.0, 'step': 22881}
Configuration saved in trained-models-notriage/distilbert-base-uncased-10565036/config.json
Model weights saved in trained-models-notriage/distilbert-base-uncased-10565036/tf_model.h5


sleeping for 30s.


100%|██████████| 1/1 [2:53:26<00:00, 10406.08s/it]

CPU times: user 2h 27min 3s, sys: 1min 15s, total: 2h 28min 18s
Wall time: 2h 53min 26s


In [82]:
#https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python

def get_prediction_proba(model, text, rounding=True):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="tf")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    #probs = outputs[0].softmax(1)
    #https://github.com/huggingface/transformers/blob/master/src/transformers/pipelines/text_classification.py
    #probs = tf.nn.softmax(outputs[0])
    pred_proba = tf.nn.softmax(outputs[0]).numpy()
    result = pred_proba
    if rounding:
        result = np.round(result*100, 1)
    #print(pred_proba)
    #pred = pred_proba.argmax(axis = 1)
    # executing argmax function to get the candidate label
    return result
#    return test_labels[np.argmax(probs)]

#https://glassboxmedicine.com/2019/05/26/classification-sigmoid-vs-softmax/
#https://dataaspirant.com/difference-between-softmax-function-and-sigmoid-function/
#https://stackoverflow.com/questions/41409248/softmax-and-sigmoid-function-for-the-output-layer
def get_prediction_proba_sigmoid(model, texts, rounding=True):
    # prepare our text into tokenized sequence
    batch_size = 64
    start = 0
    result = []
    while start < len(texts):
        text_batch = texts[start:start+batch_size]
        inputs = tokenizer(text_batch, padding=True, truncation=True, max_length=max_length, return_tensors="tf")
        #print(text_batch)
        # perform inference to our model
        outputs = model(**inputs)
        pred_proba = tf.nn.sigmoid(outputs[0]).numpy()
        if rounding:
            pred_proba = np.round(pred_proba*100, 1)
        result.extend(pred_proba)
        start += batch_size
    
    return result

def get_prediction(model, text):
    probs = get_prediction_proba(model, text)
    result = probs.argmax(axis=1)
    return result

In [83]:
print(num_labels)
for seed in seeds:
        model = TFDistilBertForSequenceClassification.from_pretrained(f"trained-models-notriage/distilbert-base-uncased-{seed}", num_labels=num_labels, local_files_only=True)
    
        print(f"seed: {seed}")
        with open(f'datasets-notriage/{seed}.pickle', 'rb') as handle:
            dataset = pickle.load(handle)
            print(f"labels: {np.unique(dataset['test_labels'])}")

            test_texts = dataset["test_texts"]
            test_labels = dataset["test_labels"]

        predictions_proba = get_prediction_proba_sigmoid(model, test_texts)#[:64])
        test_labels = test_labels#[:64]

        top1_preds = [x.argmax() for x in predictions_proba]
        correct = sum(top1_preds == test_labels)
        print(correct)
        print(correct/len(test_labels))

        n = 5
        top5_preds = [np.flip(x[-n:], 0) for x in np.argsort(predictions_proba)]
        top5_correct = 0
        missed_rows_top5 = []
        for idx, label in enumerate(test_labels):
            row_top5 = top5_preds[idx]
            if label in row_top5:
                top5_correct += 1
            else:
                missed_rows_top5.append(idx)
        print(top5_correct)
        print(top5_correct/len(test_labels))
#        break
    

loading configuration file trained-models-notriage/distilbert-base-uncased-10565036/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    "30": 

47


Some layers from the model checkpoint at trained-models-notriage/distilbert-base-uncased-10565036 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at trained-models-notriage/distilbert-base-uncased-10565036 and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infer

seed: 10565036
labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
12820
0.756431437337739
16452
0.9707340099126741


In [40]:
diff_indices = np.where(top1_preds != test_labels)
diff_indices

(array([    2,    10,    22, ..., 16943, 16945, 16946]),)

In [41]:
diff_indices[0].shape

(4128,)

In [42]:
mismatched_label_ids = [test_labels[x] for x in diff_indices[0]]
mismatched_labels_actual = [le.classes_[x] for x in mismatched_label_ids]

In [43]:
mismatched_label_ids = [top1_preds[x] for x in diff_indices[0]]
mismatched_labels_pred = [le.classes_[x] for x in mismatched_label_ids]

In [44]:
df_fails = pd.DataFrame()
df_fails["actual"] = mismatched_labels_actual
df_fails["predicted"] = mismatched_labels_pred
df_fails["pair"] = tuple(zip(mismatched_labels_actual, mismatched_labels_pred))


In [45]:
df_fails["actual"].value_counts()

General                       1060
Toolbars and Customization     277
Tabbed Browser                 262
Menus                          230
Preferences                    209
Theme                          209
Bookmarks & History            208
Shell Integration              175
Address Bar                    171
Extension Compatibility        165
File Handling                  156
Security                       152
Keyboard Navigation             92
New Tab Page                    86
Search                          81
Session Restore                 79
Installer                       67
Disability Access               64
Sync                            56
Site Identity                   42
Messaging System                37
Private Browsing                28
Firefox Accounts                24
Protections UI                  22
Migration                       22
Page Info Window                21
Downloads Panel                 19
Tours                           12
Site Permissions    

In [46]:
df_fails["predicted"].value_counts()

General                       1377
Tabbed Browser                 288
Theme                          245
Bookmarks & History            240
Address Bar                    224
Preferences                    204
Toolbars and Customization     199
Menus                          166
Security                       112
File Handling                  106
Search                          99
Keyboard Navigation             94
Session Restore                 93
New Tab Page                    92
Sync                            83
Shell Integration               76
Extension Compatibility         75
Installer                       51
Private Browsing                44
Messaging System                31
Site Identity                   30
Downloads Panel                 30
Site Permissions                22
Protections UI                  20
Migration                       17
Normandy Client                 15
Disability Access               15
PDF Viewer                      12
Tours               

In [47]:
df_fails["pair"].value_counts().head(25)

(General, Tabbed Browser)                            136
(Tabbed Browser, General)                            126
(Shell Integration, General)                         123
(File Handling, General)                             119
(Extension Compatibility, General)                   110
(Preferences, General)                               107
(Security, General)                                  102
(Menus, General)                                     102
(General, Preferences)                                90
(Toolbars and Customization, General)                 81
(Bookmarks & History, General)                        76
(General, Bookmarks & History)                        76
(General, Theme)                                      72
(General, Address Bar)                                71
(General, Menus)                                      69
(Address Bar, General)                                67
(General, File Handling)                              66
(General, Toolbars and Customiz

In [48]:
df_fails.to_csv("hf_fails.csv")

In [49]:
df_fails.head()

,actual,predicted,pair
0,Installer,General,"(Installer, General)"
1,Tabbed Browser,Toolbars and Customization,"(Tabbed Browser, Toolbars and Customization)"
2,Bookmarks & History,Menus,"(Bookmarks & History, Menus)"
3,Messaging System,New Tab Page,"(Messaging System, New Tab Page)"
4,Bookmarks & History,Address Bar,"(Bookmarks & History, Address Bar)"


In [50]:
n = 5
top5_preds = [np.flip(x[-n:], 0) for x in np.argsort(predictions_proba)]
top5_correct = 0
missed_rows_top5 = []
for idx, label in enumerate(test_labels):
    row_top5 = top5_preds[idx]
    if label in row_top5:
        top5_correct += 1
    else:
        missed_rows_top5.append(idx)
print(top5_correct)
print(top5_correct/len(test_labels))

16452
0.9707340099126741


In [51]:
diff_indices = missed_rows_top5

In [52]:
mismatched_label_ids = [test_labels[x] for x in diff_indices]
mismatched_labels_actual = [le.classes_[x] for x in mismatched_label_ids]
mismatched_label_ids = [top5_preds[x] for x in diff_indices]
mismatched_labels_pred = [tuple(le.classes_[x]) for x in mismatched_label_ids]

df_fails5 = pd.DataFrame()
df_fails5["actual"] = mismatched_labels_actual
df_fails5["predicted"] = mismatched_labels_pred
df_fails5["pair"] = tuple(zip(mismatched_labels_actual, mismatched_labels_pred))

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [53]:
test_labels

array([11, 32, 13, ..., 11, 35,  1])

In [54]:
labels_decoded = [le.classes_[x] for x in test_labels]
#labels_decoded


In [55]:
df_test = pd.DataFrame()
df_test["label"] = labels_decoded

In [56]:
df_counts = pd.DataFrame()
df_counts["total"] = df["component"].value_counts()
df_counts["test_total"] = df_test["label"].value_counts()
df_counts["fails_act"] = df_fails["actual"].value_counts()
df_counts["fails_pred"] = df_fails["predicted"].value_counts()
df_counts["total_pct"] = df_counts["total"]/df.shape[0]
df_counts["test_pct"] = df_counts["test_total"]/df_test.shape[0]
df_counts["act_pct"] = df_counts["fails_act"]/df_counts["test_total"]
df_counts["pred_pct"] = df_counts["fails_pred"]/df_counts["test_total"]
df_counts

,total,test_total,fails_act,fails_pred,total_pct,test_pct,act_pct,pred_pct
General,63598,6360,1060,1377.0,0.375256,0.375266,0.166667,0.216509
Bookmarks & History,13890,1389,208,240.0,0.081957,0.081957,0.149748,0.172786
Tabbed Browser,9677,968,262,288.0,0.057099,0.057116,0.270661,0.297521
Address Bar,7955,796,171,224.0,0.046938,0.046967,0.214824,0.281407
Preferences,6798,680,209,204.0,0.040111,0.040123,0.307353,0.300000
Toolbars and Customization,6612,661,277,199.0,0.039014,0.039002,0.419062,0.301059
Theme,6112,611,209,245.0,0.036063,0.036051,0.342062,0.400982
Sync,5745,574,56,83.0,0.033898,0.033868,0.097561,0.144599
Menus,4401,440,230,166.0,0.025968,0.025962,0.522727,0.377273
Session Restore,4125,412,79,93.0,0.024339,0.024310,0.191748,0.225728


In [57]:
type(df_fails5.iloc[0]["predicted"])

tuple

In [58]:
import ast

fail5_preds = []

df_fails5["predicted"].apply(lambda x: fail5_preds.extend(x))
#ast.literal_eval(df_fails5.iloc[0]["predicted"])
#fail5_preds
""

''

In [59]:
df_fail5_preds = pd.DataFrame()
df_fail5_preds["pred"] = fail5_preds
df_fail5_preds["pred"].value_counts()

General                       476
Toolbars and Customization    189
Tabbed Browser                167
Menus                         152
Shell Integration             125
Theme                         124
Preferences                   123
Extension Compatibility       121
Bookmarks & History           114
Address Bar                   113
File Handling                 103
Security                       92
Disability Access              72
New Tab Page                   65
Search                         54
Sync                           39
Session Restore                38
Installer                      34
Messaging System               31
Keyboard Navigation            31
Page Info Window               30
Site Identity                  26
Site Permissions               17
Tours                          16
Migration                      15
Protections UI                 13
Firefox Accounts               13
Private Browsing               12
Nimbus Desktop Client          10
about:logins  

In [60]:
df_counts5 = pd.DataFrame()
df_counts5["total"] = df["component"].value_counts()
df_counts5["test_total"] = df_test["label"].value_counts()
df_counts5["fails_act"] = df_fails5["actual"].value_counts()
df_counts5["fails_pred"] = df_fail5_preds["pred"].value_counts()
df_counts5["total_pct"] = df_counts5["total"]/df.shape[0]
df_counts5["test_pct"] = df_counts5["test_total"]/df_test.shape[0]
df_counts5["act_pct"] = df_counts5["fails_act"]/df_counts5["test_total"]
df_counts5["pred_pct"] = df_counts5["fails_pred"]/df_counts5["test_total"]
df_counts5

,total,test_total,fails_act,fails_pred,total_pct,test_pct,act_pct,pred_pct
General,63598,6360,5.0,476.0,0.375256,0.375266,0.000786,0.074843
Bookmarks & History,13890,1389,25.0,114.0,0.081957,0.081957,0.017999,0.082073
Tabbed Browser,9677,968,31.0,167.0,0.057099,0.057116,0.032025,0.172521
Address Bar,7955,796,37.0,113.0,0.046938,0.046967,0.046482,0.141960
Preferences,6798,680,30.0,123.0,0.040111,0.040123,0.044118,0.180882
Toolbars and Customization,6612,661,24.0,189.0,0.039014,0.039002,0.036309,0.285930
Theme,6112,611,17.0,124.0,0.036063,0.036051,0.027823,0.202946
Sync,5745,574,13.0,39.0,0.033898,0.033868,0.022648,0.067944
Menus,4401,440,27.0,152.0,0.025968,0.025962,0.061364,0.345455
Session Restore,4125,412,17.0,38.0,0.024339,0.024310,0.041262,0.092233


In [61]:
extended_pairs = []
for idx, actual in enumerate(mismatched_labels_actual):
    predicted_list = mismatched_labels_pred[idx]
    for predicted in predicted_list:
        extended_pairs.append((actual, predicted))

In [62]:
df_extended_pairs = pd.DataFrame()
df_extended_pairs["pair"] = extended_pairs

In [63]:
df_extended_pairs

,pair
0,"(Downloads Panel, General)"
1,"(Downloads Panel, Shell Integration)"
2,"(Downloads Panel, Bookmarks & History)"
3,"(Downloads Panel, Installer)"
4,"(Downloads Panel, Extension Compatibility)"
...,...
2475,"(Shell Integration, General)"
2476,"(Shell Integration, File Handling)"
2477,"(Shell Integration, Page Info Window)"
2478,"(Shell Integration, Menus)"


In [64]:
df_fails["actual"].value_counts()


General                       1060
Toolbars and Customization     277
Tabbed Browser                 262
Menus                          230
Preferences                    209
Theme                          209
Bookmarks & History            208
Shell Integration              175
Address Bar                    171
Extension Compatibility        165
File Handling                  156
Security                       152
Keyboard Navigation             92
New Tab Page                    86
Search                          81
Session Restore                 79
Installer                       67
Disability Access               64
Sync                            56
Site Identity                   42
Messaging System                37
Private Browsing                28
Firefox Accounts                24
Protections UI                  22
Migration                       22
Page Info Window                21
Downloads Panel                 19
Tours                           12
Site Permissions    

In [65]:
df_fails["predicted"].value_counts()

General                       1377
Tabbed Browser                 288
Theme                          245
Bookmarks & History            240
Address Bar                    224
Preferences                    204
Toolbars and Customization     199
Menus                          166
Security                       112
File Handling                  106
Search                          99
Keyboard Navigation             94
Session Restore                 93
New Tab Page                    92
Sync                            83
Shell Integration               76
Extension Compatibility         75
Installer                       51
Private Browsing                44
Messaging System                31
Site Identity                   30
Downloads Panel                 30
Site Permissions                22
Protections UI                  20
Migration                       17
Normandy Client                 15
Disability Access               15
PDF Viewer                      12
Tours               

In [66]:
df_fails.to_csv("fails5.csv")

In [67]:
df_extended_pairs["pair"].value_counts()

(Address Bar, General)                      34
(Tabbed Browser, General)                   30
(Preferences, General)                      29
(Menus, General)                            27
(Bookmarks & History, General)              25
                                            ..
(Messaging System, Disability Access)        1
(Disability Access, Installer)               1
(Security, Page Info Window)                 1
(New Tab Page, Preferences)                  1
(Enterprise Policies, Shell Integration)     1
Name: pair, Length: 726, dtype: int64

In [68]:


df_extended_pairs.to_csv("extended_pairs5.csv")



In [69]:
bug_summary = "Change MR1 upgrade onboarding to Pin then Default then Theme screens"
bug_description = """[Tracking Requested - why for this release]: Based on experiment results, the "primary browser" flow did not improve metrics as well as "pin" then "default." MR1 upgrade dialog was implemented assuming "primary" would be good enough.

There's a couple asks that are still being finalized:

    allow for 3 screens
    potentially with images, e.g., pinning, set default animation
    potentially new strings for title/subtitle/etc

If there are new strings, we might need to land them as english only?
"
"""

In [70]:
test_text = [bug_summary]#+" "+bug_description]

In [71]:
tokens = tokenizer(bug_summary, truncation=True, padding=True)

In [72]:
predictions_proba = get_prediction_proba_sigmoid(model, test_text)


['Change MR1 upgrade onboarding to Pin then Default then Theme screens']


In [73]:
#predictions_proba = get_prediction_proba_sigmoid(model, test_texts)#[:64])

top1_preds = [x.argmax() for x in predictions_proba]

n = 5
top5_preds = [np.flip(x[-n:], 0) for x in np.argsort(predictions_proba)]
top5_correct = 0
missed_rows_top5 = []
row_top5 = top5_preds[:n]


In [74]:
top1_preds

[17]

In [75]:
top5_preds

[array([17, 19, 11, 43, 41])]

In [76]:
test_text

['Change MR1 upgrade onboarding to Pin then Default then Theme screens']

In [77]:
[le.classes_[x] for x in top5_preds]

[array(['Messaging System', 'New Tab Page', 'General', 'Tours',
        'Toolbars and Customization'], dtype=object)]

In [78]:
#when summary alone is used as features/text
[predictions_proba[0][x] for x in top5_preds[0]]

[98.1, 92.1, 72.6, 41.5, 23.2]

Now the summary + description as features:

In [85]:
test_text = [bug_summary+" "+bug_description]

In [86]:
tokens = tokenizer(bug_summary, truncation=True, padding=True)

In [87]:
predictions_proba = get_prediction_proba_sigmoid(model, test_text)

In [88]:
#predictions_proba = get_prediction_proba_sigmoid(model, test_texts)#[:64])

top1_preds = [x.argmax() for x in predictions_proba]

n = 5
top5_preds = [np.flip(x[-n:], 0) for x in np.argsort(predictions_proba)]
top5_correct = 0
missed_rows_top5 = []
row_top5 = top5_preds[:n]


In [89]:
#when summary + description i used
[predictions_proba[0][x] for x in top5_preds[0]]

[99.1, 71.5, 70.0, 62.1, 18.3]

In [90]:
[le.classes_[x] for x in top1_preds]

['Messaging System']

In [91]:
top1_preds

[17]

In [92]:
top5_preds

[array([17, 19, 43, 11, 41])]